**Load the required packages and data**

In [1]:
# Load the required packages 
import numpy as np
import csv
import spacy
from collections import Counter
import math

2023-04-07 09:03:18.375984: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-07 09:03:18.376024: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-07 09:03:18.420832: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-07 09:03:19.301256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-07 09:03:19.301351: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
# Load the data 
with open("data/prototyping_sentences.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        print(row)

['sentence    structure   direct_object   indirect_object']
['The driver gave the bracelet to the grandpa', 'DO', 'bracelet', 'grandpa']
['The driver gave the grandpa the bracelet', 'PO', 'bracelet', 'grandpa']
['The veteran gave the flag to the uncle', 'DO', 'flag', 'uncle']
['The veteran gave the uncle the flag', 'PO', 'flag', 'uncle']
['The guy gave the cotton to the soldier', 'DO', 'cotton', 'soldier']
['The guy gave the soldier the cotton', 'PO', 'cotton', 'soldier']
['The astronaut gave the mustard to the boy', 'DO', 'mustard', 'boy']
['The astronaut gave the boy the mustard', 'PO', 'mustard', 'boy']
['The referee gave the clarinet to the commander', 'DO', 'clarinet', 'commander']
['The referee gave the commander the clarinet', 'PO', 'clarinet', 'commander']
['The teenager gave the pliers to the human', 'DO', 'pliers', 'human']
['The teenager gave the human the pliers', 'PO', 'pliers', 'human']
['The boy gave the loaf to the man', 'DO', 'loaf', 'man']
['The boy gave the man the l

###########################################################################################################

In [3]:
def get_sentence_structure(sentence):
    sentence_structure = None
    assert sentence_structure in {'DO', 'PO', None}
    return sentence_structure

###################################################################################################################

In [4]:
nlp = spacy.load('language_modeling_env/lib/python3.7/site-packages/en_core_web_sm/en_core_web_sm-3.5.0')

def classify_sentence_structure(sentence):
    doc = nlp(sentence)
    for token in doc:
        if token.dep_ == 'dobj':
            dobj = token
            print(dobj)
        elif token.dep_ == 'dative':
            iobj = token
            print(iobj)
        elif token.dep_ == 'pobj' and token.head.pos_ == 'ADP':
            prep = token.head
            pobj = token
    if 'dobj' in locals() and 'dative' in locals():
        return 'Double Object'
    elif 'dobj' in locals() and 'pobj' in locals():
        return 'Prepositional Object'
    else:
        return 'Unknown'

In [5]:
SENTENCE = 'The bride gave the lever to the fireman'

classify_sentence_structure(SENTENCE)

lever
to


'Prepositional Object'

# Task 1

In [6]:
def clf_stc(sentence):
    doc = nlp(sentence)
    #Find the dependency matchers
    tokens_matchers = [matchers.dep_ for matchers in doc]
    
    if 'dative' in tokens_matchers and 'dobj' in tokens_matchers:
        # Now check if it is a PO
        if 'pobj'in tokens_matchers:
            #confirm again 
            if tokens_matchers.index("dobj") < tokens_matchers.index("dative"):
                sentence_structure = "PO"
        else:
            # Check for possible DO
            tokens_matchers.index("dative") < tokens_matchers.index("dobj")
            sentence_structure = "DO"
            
    else:
        sentence_structure =  None
        
    return sentence_structure

In [7]:
# testing 
SENTENCE = 'The bride gave the lever to the fireman'

clf_stc(SENTENCE)

'PO'

In [8]:
# Extract direct objects 
def extract_direct_object(sentence):
    # split the sentence into list
    tokens = sentence.split()
    
    doc = nlp(sentence)
    tokens_matchers = [matchers.dep_ for matchers in doc]
    
    # get the position of the direct object
    dobj_index = tokens_matchers.index("dobj")
    # now find the object
    direct_object = tokens[dobj_index]
    return direct_object

In [9]:
SENTENCE = 'The bride gave the fireman the lever'
extract_direct_object(SENTENCE)

'lever'

In [10]:
# Extract indirect objects 
def extract_indirect_object(sentence):
    doc = nlp(sentence)
    tokens_matchers = [matchers.dep_ for matchers in doc]
    
    # get the sentence structure
    sentence_structure = clf_stc(sentence)
    if sentence_structure == 'DO':
        indobj_index = tokens_matchers.index('dative')
        indirect_object = doc[indobj_index]
    elif sentence_structure == 'PO':
        indobj_index = tokens_matchers.index('pobj')
        indirect_object = doc[indobj_index]
    else:
        pass

    return indirect_object

In [11]:
clf_stc("yeah and she eats people food that's another thing do you do you feed your dog people food or")

'DO'

In [12]:
extract_direct_object("yeah and she eats people food that's another thing do you do you feed your dog people food or")

'food'

In [13]:
extract_indirect_object("yeah and she eats people food that's another thing do you do you feed your dog people food or")

people

In [14]:
doc = nlp("yeah and she eats people food that's another thing do you do you feed your dog people food or")
tokens = [token.dep_ for token in doc]
tokens

['ROOT',
 'cc',
 'nsubj',
 'ccomp',
 'dative',
 'dobj',
 'nsubj',
 'relcl',
 'det',
 'attr',
 'aux',
 'nsubj',
 'aux',
 'nsubj',
 'ROOT',
 'poss',
 'compound',
 'dative',
 'dobj',
 'cc']

# Task 2

In [15]:
import nltk

In [16]:
def extract_feature_1(noun_phrase, sentence):
    return len(nltk.word_tokenize(noun_phrase))



In [17]:
SENTENCE = "The bride gave the fireman the lever"

extract_feature_1("fireman", SENTENCE)

1

In [18]:
def extract_feature_2(noun_phrase, sentence):
    pos_tags = [pos for (word, pos) in nltk.pos_tag(nltk.word_tokenize(noun_phrase))]
    return ' '.join(pos_tags)



In [19]:
extract_feature_2("the fireman", SENTENCE)

'DT NN'

In [20]:
def extract_feature_3(noun_phrase, sentence):
    words = nltk.word_tokenize(noun_phrase)
    word_freqs = Counter(words)
    total_words = len(words)
    log_mean_freq = math.log(sum(word_freqs.values()) / total_words)
    log_median_freq = math.log(sorted(word_freqs.values())[total_words // 2])
    return log_mean_freq



In [25]:
extract_feature_3("fireman", SENTENCE)

0.0

In [28]:
def concatenate_features(features1, features2):
    features = [features1, features2]
    #for i in range(len(features1)):
        #features.append(features1[i] + features2[i])
    return features

In [29]:
features1 = extract_feature_1("the fireman", SENTENCE)
features2 = extract_feature_2("the fireman", SENTENCE)

concatenate_features(features1, features2)

[2, 'DT NN']

# Task 3

In [30]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Define a list of sentences
sentences = ['This is an example sentence', 'Each sentence is converted to an embedding']

# Compute embeddings for all sentences
embeddings = model.encode(sentences)

# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)

ModuleNotFoundError: No module named 'sentence_transformers'